4장에서는 구조적 API의 핵심 추상화 개념을 알아보았음<br/>
이 장에서는 DataFrame과 DataFrame의 데이터 다루는 기능을 소개하고, 특히 DataFrame의 기본 기능을 중점적으로 다룸<br/>
DataFrame을 사용한 집계, 윈도우 함수, 조인 등의 내용은 7장과 8장에서 자세히 알아볼 것임<br/>

DataFrame은 Row 타입의 **레코드**(테이블의 로우 같은)와 각 레코드에 수행할 연산 표현식을 나타내는 여러 **컬럼**으로 구성됨<br/>
**스키마**는 각 컬럼명과 데이터 타입을 정의함<br/>
DataFrame의 **파티셔닝**은 DataFrame이나 Dataset이 클러스터에서 물리적으로 배치되는 형태를 정의함<br/>
**파티셔닝 스키마**는 파티션을 배치하는 방법을 정의함<br/>
파티셔닝의 분할 기준은 특정 컬럼이나 비결정론적(nondeterministic) 값을 기반으로 설정할 수 있음<br/>

우선 DataFrame을 생성함<br/>

In [1]:
spark

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.0.7:4042
SparkContext available as 'sc' (version = 3.1.2, master = local[*], app id = local-1640957232996)
SparkSession available as 'spark'


res0: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@a21728b


In [2]:
val df = spark.read.format("json")
    .load("Downloads/Spark-The-Definitive-Guide/data/flight-data/json/2015-summary.json")

df: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


DataFrame은 컬럼을 가지며 스키마로 컬럼을 정의함<br/>
앞 예제에서 만든 DataFrame의 스키마를 살펴보겠음<br/>

In [3]:
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



스키마는 관련된 모든 것을 하나로 묶는 역할을 함<br/>
스키마에 대해 자세히 알아보겠음<br/>

# 5.1 스키마
스키마는 DataFrame의 컬럼명과 데이터 타입을 정의함<br/>
데이터 소스에서 스키마를 얻거나 직접 정의할 수 있음<br/>

*CAUTION*<br/>
데이터를 읽기 전에 스키마를 정의해야 하는지 여부는 상황에 따라 달라짐<br/>
비정형 분석(ad-hoc analysis)에서는 스키마-온-리드가 대부분 잘 동작함 (단, csv나 json 같은 일반 텍스트 파일을 사용하면 다소 느릴 수 있음)<br/>
하지만 Long 데이터 타입을 Integer 데이터 타입으로 잘못 인식하는 등 정밀도 문제가 발생할 수 있음<br/>
따라서 운영 환경에서 추출(Extract), 변환(Transform), 적재(Load)를 수행하는 ETL 작업에 스파크를 사용한다면 직접 스키마를 정의해야 함<br/>
ETL 작업 중에 데이터 타입을 알기 힘든 csv나 json 등의 데이터 소스를 사용하는 경우, 스키마 추론 과정에서 읽어들인 샘플 데이터의 타입에 따라 스키마를 결정해버릴 수 있음<br/>

In [4]:
spark.read.format("json").load("Downloads/Spark-The-Definitive-Guide/data/flight-data/json/2015-summary.json").schema

res2: org.apache.spark.sql.types.StructType = StructType(StructField(DEST_COUNTRY_NAME,StringType,true), StructField(ORIGIN_COUNTRY_NAME,StringType,true), StructField(count,LongType,true))


스키마는 *여러 개의 StructField 타입 필드로 구성된 StructType 객체*임<br/>
StructField는 이름, 데이터 타입, 컬럼이 값이 없거나 null일 수 있는지 지정하는 Boolean 값을 가짐<br/>
필요한 경우 컬럼과 관련된 메타데이터를 지정할 수도 있음<br/>
메타데이터는 해당 컬럼과 관련된 정보이며 스파크의 머신러닝 라이브러리에서 사용함<br/>

스키마는 복합 데이터 타입인 StructType을 가질 수 있음<br/>
복합 데이터 타입은 6장에서 자세히 설명함<br/>
스파크는 런타임에 데이터 타입이 스키마의 데이터 타입과 일치하지 않으면 오류를 발생시킴<br/>
다음 코드는 DataFrame에 스키마를 만들고 적용하는 예제임<br/>

In [5]:
import org.apache.spark.sql.types.{StructField, StructType, StringType, LongType}
import org.apache.spark.sql.types.Metadata

val myManualSchema = StructType(Array(
    StructField("DEST_COUNTRY_NAME", StringType, true),
    StructField("ORIGIN_COUNTRY_NAME", StringType, true),
    StructField("count", LongType, false,
        Metadata.fromJson("{\"hello\":\"world\"}"))
))

val df = spark.read.format("json").schema(myManualSchema)
    .load("Downloads/Spark-The-Definitive-Guide/data/flight-data/json/2015-summary.json")

import org.apache.spark.sql.types.{StructField, StructType, StringType, LongType}
import org.apache.spark.sql.types.Metadata
myManualSchema: org.apache.spark.sql.types.StructType = StructType(StructField(DEST_COUNTRY_NAME,StringType,true), StructField(ORIGIN_COUNTRY_NAME,StringType,true), StructField(count,LongType,false))
df: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


스파크는 자체 데이터 타입 정보를 사용하므로 프로그래밍 언어의 데이터 타입을 스파크의 데이터 타입으로 설정할 수 없음<br/>
다음 절에서는 스키마에 정의하는 컬럼에 대해 알아보겠음<br/>

# 5.2 컬럼과 표현식
스파크의 컬럼은 스프레드시트, R의 dataframe, Pandas의 DataFrame 컬럼과 유사함<br/>
사용자는 **표현식**으로 DataFrame의 컬럼을 선택, 조작, 제거할 수 있음<br/>

스파크의 컬럼은 *표현식을 사용해 레코드 단위로 계산한 값을 단순하게 나타내는 논리적인 구조*임<br/>
따라서 컬럼의 실제 값을 얻으려면 로우가 필요하고, 로우를 얻으려면 DataFrame이 필요함<br/>
DataFrame을 통하지 않으면 외부에서 컬럼에 접근할 수 없음<br/>
컬럼 내용을 수정하려면 반드시 DataFrame의 스파크 transformation을 사용해야 함<br/>

## 5.2.1 컬럼
컬럼을 생성하고 참조할 수 있는 여러 가지 방법이 있지만, col 함수나 column 함수를 사용하는 것이 가장 간단함<br/>
이들 함수는 컬럼명을 인수로 받음<br/>

In [6]:
import org.apache.spark.sql.functions.{col, column}

col("someColumnName")
column("someColumnName")

import org.apache.spark.sql.functions.{col, column}
res3: org.apache.spark.sql.Column = someColumnName


이 책에서는 col 함수를 계속해서 사용함<br/>
컬럼이 DataFrame에 있을지 없을지는 알 수 없음<br/>
컬럼은 컬럼명을 **카탈로그**에 저장된 정보와 비교하기 전까지 **미확인** 상태로 남음<br/>
4장에서 알아본 것처럼 **분석기**가 동작하는 단계에서 컬럼과 테이블을 분석함<br/>

### 명시적 컬럼 참조
DataFrame의 컬럼은 col 메서드로 참조함<br/>
col 메서드는 조인 시 유용함<br/>
예를 들어 DataFrame의 어떤 컬럼을 다른 DataFrame의 조인 대상 컬럼에서 참조하기 위해 col 메서드를 사용함<br/>
조인은 8장에서 자세히 알아보겠음<br/>
col 메서드를 사용해 명시적으로 컬럼을 정의하면 스파크는 분석기 실행 단계에서 컬럼 확인 절차를 생략함<br/>

In [7]:
df.col("count")

res4: org.apache.spark.sql.Column = count


## 5.2.2 표현식

### 표현식으로 컬럼 표현

In [8]:
(((col("someCol") + 5) * 200) - 6) < col("otherCol")

res5: org.apache.spark.sql.Column = ((((someCol + 5) * 200) - 6) < otherCol)


In [9]:
import org.apache.spark.sql.functions.expr

expr("(((someCol + 5) * 200) - 6) < otherCol")

import org.apache.spark.sql.functions.expr
res6: org.apache.spark.sql.Column = ((((someCol + 5) * 200) - 6) < otherCol)


### DataFrame 컬럼에 접근하기

In [11]:
spark.read.format("json").load("Downloads/Spark-The-Definitive-Guide/data/flight-data/json/2015-summary.json").columns

res8: Array[String] = Array(DEST_COUNTRY_NAME, ORIGIN_COUNTRY_NAME, count)


# 5.3 레코드와 로우

In [12]:
df.first()

res9: org.apache.spark.sql.Row = [United States,Romania,15]


## 5.3.1 로우 생성하기

In [13]:
import org.apache.spark.sql.Row

val myRow = Row("Hello", null, 1, false)

import org.apache.spark.sql.Row
myRow: org.apache.spark.sql.Row = [Hello,null,1,false]


In [18]:
myRow(0) // Any 타입

res14: Any = Hello


In [19]:
myRow(0).asInstanceOf[String] // String 타입

res15: String = Hello


In [20]:
myRow.getString(0) // String 타입

res16: String = Hello


In [21]:
myRow.getInt(2) // Int 타입

res17: Int = 1


Dataset API를 이용하면 JVM 객체를 가진 데이터셋을 얻을 수 있음<br/>
Dataset은 11장에서 자세히 알아보겠음<br/>

# 5.4 DataFrame의 트랜스포메이션

지금까지 DataFrame의 핵심 영역을 간단히 살펴봤음 <br/>
이어서 DataFrame을 다루는 방법을 알아보겠음<br/>
DataFrame을 다루는 방법은 몇 가지 주요 작업으로 나눌 수 있음<br/>
* 로우나 컬럼 추가
* 로우나 컬럼 제거
* 로우를 컬럼으로 변환하거나, 그 반대로 변환
* 컬럼값을 기준으로 로우 순서 변경

## 5.4.1 DataFrame 생성하기

In [22]:
val df = spark.read.format("json")
    .load("Downloads/Spark-The-Definitive-Guide/data/flight-data/json/2015-summary.json")
df.createOrReplaceTempView("dfTable")

df: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


In [23]:
import org.apache.spark.sql.Row
import org.apache.spark.sql.types.{StructField, StructType, StringType, LongType}

val myManualSchema = new StructType(Array(
    new StructField("some", StringType, true),
    new StructField("col", StringType, true),
    new StructField("names", LongType, false)))

val myRows = Seq(Row("Hello", null, 1L))
val myRDD = spark.sparkContext.parallelize(myRows)
val myDf = spark.createDataFrame(myRDD, myManualSchema)

myDf.show()

+-----+----+-----+
| some| col|names|
+-----+----+-----+
|Hello|null|    1|
+-----+----+-----+



import org.apache.spark.sql.Row
import org.apache.spark.sql.types.{StructField, StructType, StringType, LongType}
myManualSchema: org.apache.spark.sql.types.StructType = StructType(StructField(some,StringType,true), StructField(col,StringType,true), StructField(names,LongType,false))
myRows: Seq[org.apache.spark.sql.Row] = List([Hello,null,1])
myRDD: org.apache.spark.rdd.RDD[org.apache.spark.sql.Row] = ParallelCollectionRDD[23] at parallelize at <console>:40
myDf: org.apache.spark.sql.DataFrame = [some: string, col: string ... 1 more field]


*NOTE*<br/>
스칼라 버전의 스파크 콘솔을 사용하는 경우 Seq 데이터 타입에 toDF 함수를 활용할 수 있어 스파크의 implicits가 주는 장점을 얻을 수 있음<br/>
물론 implicits를 import해야 함<br/>
하지만 implicits는 null 타입과 잘 맞지 않음<br/>
그러므로 실제 운영 환경에서 사용하는 것은 권장하지 않음<br/>

In [24]:
val myDF = Seq(("Hello", 2, 1L)).toDF("col1", "col2", "col3")

myDF: org.apache.spark.sql.DataFrame = [col1: string, col2: int ... 1 more field]


지금까지 DataFrame을 만드는 방법을 알아보았음<br/>
이제 다음과 같이 가장 유용하게 사용할 수 있는 메서드를 알아보겠음<br/>
* 컬럼이나 표현식을 사용하는 select 메서드
* 문자열 표현식을 사용하는 selectExpr 메서드
* 메서드로 사용할 수 없는 org.apache.spark.sql.functions 패키지에 포함된 다양한 함수

이 3가지 유형의 메서드로 DataFrame을 다룰 때 필요한 대부분의 transformation 작업을 해결할 수 있음<br/>

## 5.4.2 select와 selectExpr

In [26]:
df.select("DEST_COUNTRY_NAME").show(2)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
+-----------------+
only showing top 2 rows



In [27]:
df.select("DEST_COUNTRY_NAME", "ORIGIN_COUNTRY_NAME").show(2)

+-----------------+-------------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|
+-----------------+-------------------+
|    United States|            Romania|
|    United States|            Croatia|
+-----------------+-------------------+
only showing top 2 rows



In [34]:
import org.apache.spark.sql.functions.{expr, col, column}

df.select(df.col("DEST_COUNTRY_NAME")).show(2)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
+-----------------+
only showing top 2 rows



import org.apache.spark.sql.functions.{expr, col, column}


In [35]:
df.select(col("DEST_COUNTRY_NAME")).show(2)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
+-----------------+
only showing top 2 rows



In [36]:
df.select(column("DEST_COUNTRY_NAME")).show(2)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
+-----------------+
only showing top 2 rows



In [37]:
df.select('DEST_COUNTRY_NAME).show(2)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
+-----------------+
only showing top 2 rows



In [38]:
df.select($"DEST_COUNTRY_NAME").show(2)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
+-----------------+
only showing top 2 rows



In [39]:
df.select(expr("DEST_COUNTRY_NAME")).show(2)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
+-----------------+
only showing top 2 rows



Column 객체와 문자열을 함께 섞어 쓰는 실수를 많이 함<br/>
다음 코드 예제를 실행하면 컴파일러 오류가 발생함<br/>

In [40]:
df.select(col("DEST_COUNTRY_NAME"), "DEST_COUNTRY_NAME")

<console>: 37: error: overloaded method value select with alternatives:

expr 함수는 가장 유연한 참조 방법임<br/>
expr 함수는 단순 컬럼 참조나 문자열을 이용해 컬럼을 참조할 수 있음<br/>
설명을 위해 AS 키워드로 컬럼명을 변경한 다음 alias 메서드로 원래 컬럼명으로 되돌려보겠음<br/>

In [41]:
df.select(expr("DEST_COUNTRY_NAME AS destination")).show(2)

+-------------+
|  destination|
+-------------+
|United States|
|United States|
+-------------+
only showing top 2 rows



아래 코드는 변경함 컬럼명을 원래 이름으로 되돌려 놓음<br/>

In [43]:
df.select(expr("DEST_COUNTRY_NAME AS destination").alias("DEST_COUNTRY_NAME")).show(2)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
+-----------------+
only showing top 2 rows



우리는 select 메서드에 expr 함수를 사용하는 패턴을 자주 활용함<br/>
스파크는 이런 작업을 간단하고 효율적으로 할 수 있는 *selectExpr* 메서드를 제공함<br/>
*selectExpr* 메서드는 자주 사용하는 편리한 인터페이스 중 하나임<br/>

In [44]:
df.selectExpr("DEST_COUNTRY_NAME AS newColumnName", "DEST_COUNTRY_NAME").show(2)

+-------------+-----------------+
|newColumnName|DEST_COUNTRY_NAME|
+-------------+-----------------+
|United States|    United States|
|United States|    United States|
+-------------+-----------------+
only showing top 2 rows



selectExpr 메서드는 스파크의 진정한 능력을 보여줌<br/>
selectExpr 메서드는 새로운 DataFrame을 생성하는 복잡한 표현식을 간단하게 만드는 도구임<br/>
사실 모든 유효한 비집계형(non-aggregating) SQL 구문을 지정할 수 있음<br/>
단, 컬럼을 식별할 수 있어야 함<br/>
다음 코드는 DataFrame에 출발지와 도착지가 같은지 나타내는 새로운 withinCountry 컬럼을 추가하는 예제임<br/>

In [52]:
df.selectExpr(
    "*", // 모든 원본 컬럼 포함
    "(DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME) as withinCountry")
    .show(85)

+--------------------+--------------------+------+-------------+
|   DEST_COUNTRY_NAME| ORIGIN_COUNTRY_NAME| count|withinCountry|
+--------------------+--------------------+------+-------------+
|       United States|             Romania|    15|        false|
|       United States|             Croatia|     1|        false|
|       United States|             Ireland|   344|        false|
|               Egypt|       United States|    15|        false|
|       United States|               India|    62|        false|
|       United States|           Singapore|     1|        false|
|       United States|             Grenada|    62|        false|
|          Costa Rica|       United States|   588|        false|
|             Senegal|       United States|    40|        false|
|             Moldova|       United States|     1|        false|
|       United States|        Sint Maarten|   325|        false|
|       United States|    Marshall Islands|    39|        false|
|              Guyana|   

select 표현식에는 DataFrame의 컬럼에 대한 집계 함수를 지정할 수 있음<br/>
다음 예제는 지금까지의 예제와 크게 다르지 않음<br/>

In [58]:
df.selectExpr("avg(count)", "count(distinct(DEST_COUNTRY_NAME))").show() // show(1)이든 show(100)이든 결과는 같음

+-----------+---------------------------------+
| avg(count)|count(DISTINCT DEST_COUNTRY_NAME)|
+-----------+---------------------------------+
|1770.765625|                              132|
+-----------+---------------------------------+



## 5.4.3 스파크 데이터 타입으로 변환하기

때로는 새로운 컬럼이 아닌 명시적인 값을 스파크에 전달해야 함<br/>
명시적인 값은 상수값일 수 있고, 추후 비교에 사용할 무언가가 될 수도 있음<br/>
이때 **리터럴(literal)**을 사용함<br/>
리터럴은 프로그래밍 언어의 리터럴 값을 스파크가 이해할 수 있는 값으로 변환함<br/>
리터럴은 표현식이며 이전 예제와 같은 방식으로 사용함<br/>

In [61]:
import org.apache.spark.sql.functions.lit

df.select(expr("*"), lit(1).as("One")).show(2)

+-----------------+-------------------+-----+---+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|One|
+-----------------+-------------------+-----+---+
|    United States|            Romania|   15|  1|
|    United States|            Croatia|    1|  1|
+-----------------+-------------------+-----+---+
only showing top 2 rows



import org.apache.spark.sql.functions.lit


어떤 상수나 프로그래밍으로 생성된 변숫값이 특정 컬럼의 값보다 큰지 확인할 때 리터럴을 사용함<br/>

## 5.4.4 컬럼 추가하기

DataFrame에 신규 컬럼을 추가하는 공식적인 방법은 DataFrame의 withColumn 메서드를 사용하는 것임<br/>
숫자 1을 값으로 가지는 컬럼을 추가하는 예제는 다음과 같음<br/>

In [62]:
df.withColumn("numberOne", lit(1)).show(2)

+-----------------+-------------------+-----+---------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|numberOne|
+-----------------+-------------------+-----+---------+
|    United States|            Romania|   15|        1|
|    United States|            Croatia|    1|        1|
+-----------------+-------------------+-----+---------+
only showing top 2 rows



다음은 출발지와 도착지가 같은지 여부를 Boolean 타입으로 표현하는 예제임<br/>

In [68]:
df.withColumn("withinCountry", expr("ORIGIN_COUNTRY_NAME == DEST_COUNTRY_NAME")).show(2) // expr 안에 =을 써도 ==을 써도 같은 결과가 나옴

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



withColumn 메서드는 2개의 인수를 사용함<br/>
하나는 컬럼명이고, 다른 하나는 값을 생성할 표현식임<br/>
한 가지 재미있는 것은 withColumn 메서드로 컬럼명을 변경할 수도 있다는 것임<br/>

In [69]:
df.withColumn("Destination", expr("DEST_COUNTRY_NAME")).columns

res62: Array[String] = Array(DEST_COUNTRY_NAME, ORIGIN_COUNTRY_NAME, count, Destination)


## 5.4.5 컬럼명 변경하기

withColumn 메서드 대신 withColumnRenamed 메서드로 컬럼명을 변경할 수도 있음<br/>
withColumnRenamed 메서드는 첫 번째 인수로 전달된 컬럼명을 두 번째 인수의 문자열로 변경함<br/>

In [70]:
df.withColumnRenamed("DEST_COUNTRY_NAME", "dest").columns

res63: Array[String] = Array(dest, ORIGIN_COUNTRY_NAME, count)


## 5.4.6 예약 문자와 키워드

공백이나 하이픈(-) 같은 예약 문자는 컬럼명에 사용할 수 없음<br/>
예약 문자를 컬럼명에 사용하려면 백틱(`) 문자를 이용해 escaping해야 함<br/>
withColumn 메서드를 사용해 예약 문자가 포함된 컬럼을 생성해보겠음<br/>
다음은 escaping 문자가 필요한 경우와 필요 없는 경우의 예제임<br/>

In [71]:
import org.apache.spark.sql.functions.expr

val dfWithLongColName = df.withColumn(
    "This Long Column-Name",
    expr("ORIGIN_COUNTRY_NAME"))

import org.apache.spark.sql.functions.expr
dfWithLongColName: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 2 more fields]


위 예제에서는 withColumn 메서드의 첫 번째 인수로 새로운 컬럼명을 나타내는 문자열을 지정했기 때문에 escape 문자가 필요 없음<br/>
하지만 다음 예제에서는 표현식으로 컬럼을 참조하므로 백틱(`) 문자를 사용함<br/>

In [72]:
dfWithLongColName.selectExpr(
    "`This Long Column-Name`",
    "`This Long Column-Name` as `new col`")
    .show(2)

+---------------------+-------+
|This Long Column-Name|new col|
+---------------------+-------+
|              Romania|Romania|
|              Croatia|Croatia|
+---------------------+-------+
only showing top 2 rows



표현식 대신 문자열을 사용해 명시적으로 컬럼을 참조하면 리터럴로 해석되기 때문에 예약 문자가 포함된 컬럼을 참조할 수 있음<br/>
예약 문자나 키워드를 사용하는 표현식에만 escape 처리가 필요함<br/>

In [73]:
dfWithLongColName.select(col("This Long Column-Name")).columns

res65: Array[String] = Array(This Long Column-Name)


## 5.4.7 대소문자 구분

## 5.4.8 컬럼 제거하기

기본적으로 스파크는 대소문자를 가리지 않음<br/>
다음과 같은 설정을 사용해 스파크에서 대소문자를 구분하게 만들 수 있음<br/>

DataFrame에서 컬럼을 제거하는 방법을 알아보겠음<br/>
select 메서드로 컬럼을 제거할 수 있지만 컬럼을 제거하는 메서드인 drop을 사용할 수도 있음<br/>

In [74]:
df.drop("ORIGIN_COUNTRY_NAME").columns

res66: Array[String] = Array(DEST_COUNTRY_NAME, count)


다수의 컬럼명을 drop 메서드의 인수로 사용해 컬럼을 한꺼번에 제거할 수 있음<br/>

In [76]:
dfWithLongColName.drop("ORIGIN_COUNTRY_NAME", "DEST_COUNTRY_NAME").columns

res68: Array[String] = Array(count, This Long Column-Name)


## 5.4.9 컬럼의 데이터 타입 변경하기

가끔 특정 데이터 타입을 다른 데이터 타입으로 형변환할 필요가 있음<br/>
다수의 StringType 컬럼을 정수형으로 변환해야 하는 경우가 그 예임<br/>
cast 메서드로 데이터 타입을 변환할 수 있음<br/>
다음은 count 컬럼을 Integer 데이터 타입에서 String 데이터 타입으로 형변환하는 예제임<br/>

In [77]:
df.withColumn("count2", col("count").cast("string"))

res69: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 2 more fields]


## 5.4.10 로우 필터링하기

로우를 필터링하려면 참과 거짓을 판별하는 표현식을 만들어야 함<br/>
그러면 표현식의 결과가 false인 로우를 걸러낼 수 있음<br/>
*DataFrame의 가장 일반적인 필터링 방법은 문자열 표현식이나 컬럼을 다루는 기능을 이용해 표현식을 만드는 것임<br/>
DataFrame의 where 메서드나 filter 메서드로 필터링할 수 있음*<br/>
이 두 메서드 모두 같은 연산을 수행하며 같은 파라미터 타입을 사용함<br/>
이 중 SQL과 유사한 where 메서드를 앞으로 계속 사용하겠지만, filter도 사용할 수 있다는 점을 기억하기 바람<br/>

*NOTE*<br/>
스칼라나 자바에서 Dataset API를 이용해 filter 메서드를 사용하면 Dataset의 각 레코드에 적용할 함수를 filter 메서드에 사용할 수 있음<br/>
자세한 내용을 11장을 참조할 것<br/>

다음 예제의 filter와 where 메소드는 모두 동일하게 동작함<br/>
스칼라와 파이썬 모두 같은 결과를 반환함<br/>

In [78]:
df.filter(col("count") < 2).show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



In [79]:
df.where("count < 2").show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



같은 표현식에 여러 필터를 적용해야 할 때도 있음<br/>
하지만 스파크는 자동으로 필터의 순서와 상관없이 동시에 모든 필터링 작업을 수행하기 때문에 항상 유용한 것은 아님<br/>
그러므로 여러 개의 AND 필터를 지정하려면 차례대로 필터를 연결하고 판단은 스파크에 맡겨야 함<br/>

In [80]:
df.where(col("count") < 2).where(col("ORIGIN_COUNTRY_NAME") =!= "Croatia").show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|          Singapore|    1|
|          Moldova|      United States|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



## 5.4.11 고유한 로우 얻기

일반적으로 DataFrame에서 고윳값이나 중복되지 않은 값을 얻는 연산을 자주 사용함<br/>
DataFrame의 모든 로우에서 중복 데이터를 제거할 수 있는 distinct 메서드를 사용해 고윳값을 찾을 수 있음<br/>
항공운항 데이터셋에서 중복되지 않은 출발지 정보를 얻는 예제를 살펴보겠음<br/>
distinct 메서드는 중복되지 않은 로우를 가진 신규 DataFrame을 반환함<br/>

In [81]:
df.select("ORIGIN_COUNTRY_NAME", "DEST_COUNTRY_NAME").distinct().count()

res73: Long = 256


In [82]:
df.select("ORIGIN_COUNTRY_NAME").distinct().count()

res74: Long = 125


## 5.4.12 무작위 샘플 만들기

DataFrame에서 무작위 샘플 데이터를 얻으려면 DataFrame의 sample 메서드를 사용함<br/>
DataFrame에서 표본 데이터 추출 비율을 지정할 수 있으며, 복원 추출이나 비복원 추출의 사용 여부를 지정할 수도 있음<br/>

In [86]:
val seed = 5
val withReplacement = false
val fraction = 0.5
df.sample(withReplacement, fraction, seed).count()

seed: Int = 5
withReplacement: Boolean = false
fraction: Double = 0.5
res78: Long = 138


## 5.4.13 임의 분할하기

임의 분할(random split)은 원본 DataFrame을 임의 크기로 '분할'할 때 유용하게 사용됨<br/>
이 기능은 머신러닝 알고리즘에서 사용할 학습셋, 검증셋, 그리고 테스트셋을 만들 때 주로 사용함<br/>
다음 예제에서는 분할 가중치를 함수의 파라미터로 설정해 원본 DataFrame을 서로 다른 데이터를 가진 두 개의 DataFrame으로 나눔<br/>
이 메서드는 임의성을 가지도록 설계되었으므로 시드값을 반드시 설정해야 함<br/>
총합이 1이 되도록 각 DataFrame의 비율을 지정하지 않으면 예제와 같은 비율로 지정됨<br/>

In [87]:
val dataFrames = df.randomSplit(Array(0.25, 0.75), seed)
dataFrames(0).count() > dataFrames(1).count() // False

dataFrames: Array[org.apache.spark.sql.Dataset[org.apache.spark.sql.Row]] = Array([DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field], [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field])
res79: Boolean = false


*보충 설명*

번역본이어서 매끄럽지 않은 부분도 있고 이 부분에 대한 책 설명이 다소 부족하다고 느껴져서 [여기](https://sparkbyexamples.com/spark/spark-sampling-with-examples/)를 참고한 내용을 덧붙여보겠음<br/>


In [88]:
val anotherDF = spark.range(100)
println(anotherDF.collect().mkString(","))

0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99


anotherDF: org.apache.spark.sql.Dataset[Long] = [id: bigint]


우선 fraction은 dataFrame에서 대략 어느 정도 비율만큼 샘플링을 할 것이냐를 의미하는 파라미터임<br/>
하지만 fraction을 지정한다고 딱 그만큼의 비율을 샘플링하는 것은 아니고 대략 그 정도만큼을 샘플링하게 됨<br/>
아래 예시를 보면 같은 dataFrame에 대해 같은 fraction 값을 썼지만 출력되는 값의 개수는 다른 것을 볼 수 있음<br/>

In [89]:
println(anotherDF.sample(0.1).collect().mkString(","))

7,33,56,66,69,76


In [90]:
println(anotherDF.sample(0.1).collect().mkString(","))

10,14,33,40,43,63,65,79,87,92


seed의 경우 sample 메서드로 샘플링된 결과를 여러 번 부를 때 sample 메서드 자체의 임의성에도 불구하고 항상 같은 결과를 얻기 위해 사용하는 것임<br/>
즉 seed 값을 같게 하면 sample 메서드로 샘플링된 결과가 같게 함으로써 임의성을 가진 sample 메서드로부터 항상 같은 결과를 얻는 것임<br/>
물론 seed 값을 다르게 하면 다른 결과를 얻게 됨<br/>

In [92]:
println(anotherDF.sample(0.1, 2).collect().mkString(","))

9,26,29,33,35,37,38,42,46,48,50,61,73,85,89


In [93]:
println(anotherDF.sample(0.1, 2).collect().mkString(",")) // 같은 시드값을 쓰기 때문에 위와 같은 샘플링 결과를 얻음

9,26,29,33,35,37,38,42,46,48,50,61,73,85,89


In [94]:
println(anotherDF.sample(0.1, 37).collect().mkString(",")) // 다른 시드값을 쓰기 때문에 위와 다른 샘플링 결과를 얻음

11,19,21,30,38,41,56,69,84


## 5.4.14 로우 합치기와 추가하기

앞서 배웠듯이 DataFrame은 불변성을 가짐<br/>
그러므로 DataFrame에 레코드를 추가하는 작업은 DataFrame을 변경하는 작업이기 때문에 불가능함<br/>
DataFrame에 레코드를 추가하려면 원본 DataFrame을 새로운 DataFrame과 **통합(union)**해야 함<br/>
통합은 두 개의 DataFrame을 단순히 결합하는 행위임<br/>
통합하려는 두 개의 DataFrame은 반드시 동일한 스키마와 컬럼 수를 가져야 함<br/>

*CAUTION*<br/>
union 메서드는 현재 스키마가 아닌 컬럼 위치를 기반으로 동작함<br/>
따라서 사용자가 생각한 대로 자동 정렬되지 않을 수도 있음<br/>
-> 이 부분은 아직 무슨 뜻인지 잘 이해가 안 됨<br/>

In [98]:
import org.apache.spark.sql.Row

val schema = df.schema

val newRows = Seq(
    Row("New Country", "Other Country", 5L), // 이 신규 로우는 5L이므로 count값이 5
    Row("New Country 2", "Other Country 3", 1L) // 이 신규 로우는 1L이므로 count값이 1
)

val parallelizedRows = spark.sparkContext.parallelize(newRows)
val newDF = spark.createDataFrame(parallelizedRows, schema)

df.union(newDF)
    .where("count = 1")
    .where($"ORIGIN_COUNTRY_NAME" =!= "United States")
    .show() // 전체 데이터를 조회하면 신규 로우를 확인할 수 있음 

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
|    United States|          Gibraltar|    1|
|    United States|             Cyprus|    1|
|    United States|            Estonia|    1|
|    United States|          Lithuania|    1|
|    United States|           Bulgaria|    1|
|    United States|            Georgia|    1|
|    United States|            Bahrain|    1|
|    United States|   Papua New Guinea|    1|
|    United States|         Montenegro|    1|
|    United States|            Namibia|    1|
|    New Country 2|    Other Country 3|    1|
+-----------------+-------------------+-----+



import org.apache.spark.sql.Row
schema: org.apache.spark.sql.types.StructType = StructType(StructField(DEST_COUNTRY_NAME,StringType,true), StructField(ORIGIN_COUNTRY_NAME,StringType,true), StructField(count,LongType,true))
newRows: Seq[org.apache.spark.sql.Row] = List([New Country,Other Country,5], [New Country 2,Other Country 3,1])
parallelizedRows: org.apache.spark.rdd.RDD[org.apache.spark.sql.Row] = ParallelCollectionRDD[328] at parallelize at <console>:58
newDF: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


로우가 추가된 DataFrame을 참조하려면 새롭게 만들어진 DataFrame 객체(예제 기준으로는 newDF)를 사용해야 함<br/>
DataFrame을 뷰로 만들거나 테이블로 등록하면 DataFrame 변경 작업과 관계없이 동적으로 참조할 수 있음<br/>

## 5.4.15 로우 정렬하기

sort와 orderBy 메서드를 사용해 DataFrame의 최댓값 혹은 최솟값이 상단에 위치하도록 정렬할 수 있음<br/>
두 메서드는 완전히 같은 방식으로 동작함 (스파크 코드를 살펴보면 orderBy 메서드 내부에서 sort 메서드를 호출함)<br/>
두 메서드 모두 컬럼 표현식과 문자열을 사용할 수 있으며 다수의 컬럼을 지정할 수 있음<br/>
*기본 동작 방식은 오름차순 정렬임*<br/>

In [103]:
df.sort("count").show(5)

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|               Malta|      United States|    1|
|Saint Vincent and...|      United States|    1|
|       United States|            Croatia|    1|
|       United States|          Gibraltar|    1|
|       United States|          Singapore|    1|
+--------------------+-------------------+-----+
only showing top 5 rows



In [104]:
df.orderBy("count", "DEST_COUNTRY_NAME").show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|     Burkina Faso|      United States|    1|
|    Cote d'Ivoire|      United States|    1|
|           Cyprus|      United States|    1|
|         Djibouti|      United States|    1|
|        Indonesia|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



In [105]:
df.orderBy(col("count"), col("DEST_COUNTRY_NAME")).show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|     Burkina Faso|      United States|    1|
|    Cote d'Ivoire|      United States|    1|
|           Cyprus|      United States|    1|
|         Djibouti|      United States|    1|
|        Indonesia|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



정렬 기준을 명확히 지정하려면 asc나 desc 함수를 사용함<br/>
두 함수 모두 컬럼의 정렬 순서를 지정함<br/>

In [107]:
df.orderBy(desc("count"), asc("DEST_COUNTRY_NAME")).show(2)

+-----------------+-------------------+------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|
+-----------------+-------------------+------+
|    United States|      United States|370002|
|    United States|             Canada|  8483|
+-----------------+-------------------+------+
only showing top 2 rows



asc_nulls_first, desc_nulls_first, asc_nulls_last, desc_nulls_last 메서드를 사용하여 정렬된 DataFrame에서 null 값이 표시되는 기준을 지정할 수 있음 <br/>

transformation을 처리하기 전에 성능을 최적화하기 위해 파티션별 정렬을 수행하기도 함<br/>
파티션별 정렬은 sortWithinPartitions 메서드로 할 수 있음<br/>

In [111]:
spark.read.format("json").load("Downloads/Spark-The-Definitive-Guide/data/flight-data/json/*-summary.json")
    .sortWithinPartitions("count")

res103: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


튜닝과 최적화 내용은 3부(저수준 API; 12~14장)에서 자세히 알아보겠음<br/>

## 5.4.16 로우 수 제한하기

DataFrame에서 추출할 로우 수를 제한해야 할 때가 있음<br/>
DataFrame에서 상위 10개의 결과만을 보고자 하는 경우가 그 예임<br/>
limit 메서드를 사용해 추출할 로우 수를 제한할 수 있음<br/>

In [112]:
df.limit(5).show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+



In [113]:
df.orderBy(desc("count")).limit(6).show()

+-----------------+-------------------+------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|
+-----------------+-------------------+------+
|    United States|      United States|370002|
|    United States|             Canada|  8483|
|           Canada|      United States|  8399|
|    United States|             Mexico|  7187|
|           Mexico|      United States|  7140|
|   United Kingdom|      United States|  2025|
+-----------------+-------------------+------+



## 5.4.17 repartition과 coalesce

또 다른 최적화 기법은 자주 필터링하는 컬럼을 기준으로 데이터를 분할하는 것임<br/>
이를 통해 파티셔닝 스키마와 파티션 수를 포함해 클러스터 전반의 물리적인 데이터 구성을 제어할 수 있음<br/>

repartition 메서드를 호출하면 무조건 전체 데이터를 셔플함<br/>
향후에 사용할 파티션 수가 현재 파티션 수보다 많거나 컬럼을 기준으로 파티션을 만드는 경우에만 사용해야 함 -> 왜? 잘 이해가 안 됨<br/>

In [114]:
df.rdd.getNumPartitions // 1

res106: Int = 1


In [115]:
df.repartition(5)

res107: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


특정 컬럼을 기준으로 자주 필터링한다면 자주 필터링되는 컬럼을 기준으로 파티션을 재분배하는 것이 좋음

In [116]:
df.repartition(col("DEST_COUNTRY_NAME"))

res108: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


선택적으로 파티션 수를 지정할 수도 있음

In [117]:
df.repartition(5, col("DEST_COUNTRY_NAME"))

res109: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


coalesce 메서드는 전체 데이터를 셔플하지 않고 파티션을 병합하려는 경우에 사용함<br/>
(파티션 수를 즐이려면 셔플이 일어나는 repartition 대신 coalesce를 사용해야 함)<br/>
다음은 목적지를 기준으로 셔플을 수행해 5개의 파티션으로 나누고, 전체 데이터를 셔플 없이 병합하는 예제임<br/>

*의문사항<br/>
그냥 셔플을 수행하는 것과 목적지(특정 컬럼)를 기준으로 셔플을 수행하는 것의 차이점?<br/>

In [118]:
df.repartition(5, col("DEST_COUNTRY_NAME")).coalesce(2)

res110: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


## 5.4.18 드라이버로 로우 데이터 수집하기

스파크는 드라이버에서 클러스터 상태 정보를 유지함<br/>
로컬 환경에서 데이터를 다루려면 드라이버로 데이터를 수집해야 함<br/>

아직 드라이버로 데이터를 수집하는 연산을 정확하게 설명하지 않았음<br/>
하지만 몇 가지 메서드는 이미 사용해보았음<br/>
collect 메서드는 전체 DataFrame의 모든 데이터를 수집하며, take 메서드는 상위 N개의 로우를 반환함<br/>
show 메서드는 여러 로우를 보기 좋게 출력함<br/>

In [120]:
val collectDF = df.limit(10)
collectDF.take(5) // take는 정수형 값을 인수로 사용함

collectDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]
res112: Array[org.apache.spark.sql.Row] = Array([United States,Romania,15], [United States,Croatia,1], [United States,Ireland,344], [Egypt,United States,15], [United States,India,62])


In [121]:
collectDF.show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
|    United States|          Singapore|    1|
|    United States|            Grenada|   62|
|       Costa Rica|      United States|  588|
|          Senegal|      United States|   40|
|          Moldova|      United States|    1|
+-----------------+-------------------+-----+



In [122]:
collectDF.show(5, false)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|United States    |Romania            |15   |
|United States    |Croatia            |1    |
|United States    |Ireland            |344  |
|Egypt            |United States      |15   |
|United States    |India              |62   |
+-----------------+-------------------+-----+
only showing top 5 rows



In [123]:
collectDF.collect()

res115: Array[org.apache.spark.sql.Row] = Array([United States,Romania,15], [United States,Croatia,1], [United States,Ireland,344], [Egypt,United States,15], [United States,India,62], [United States,Singapore,1], [United States,Grenada,62], [Costa Rica,United States,588], [Senegal,United States,40], [Moldova,United States,1])


전체 데이터셋에 대한 반복(iterate) 처리를 위해 드라이버로 로우를 모으는 또 다른 방법이 있음<br/>
toLocalIterator 메서드는 이터레이터(iterator)로 모든 파티션의 데이터를 드라이버에 전달함<br/>
toLocalIterator 메서드를 사용해 데이터셋의 파티션을 차례로 반복 처리할 수 있음<br/>

In [124]:
collectDF.toLocalIterator()

res116: java.util.Iterator[org.apache.spark.sql.Row] = IteratorWrapper(<iterator>)


*CAUTION*<br/>
드라이버로 모든 데이터 컬렉션을 수집하는 작업은 매우 큰 비용(CPU, 메모리, 네트워크 등)이 발생함<br/>
대규모 데이터셋에 collect 명령을 수행하면 드라이버가 비정상적으로 종료될 수 있음<br/>
toLocalIterator 메서드도 마찬가지임<br/>
toLocalIterator 메서드를 사용할 때 매우 큰 파티션이 있다면 드라이버와 애플리케이션이 비정상적으로 종료될 수 있음<br/>
또한 연산을 병렬로 수행하지 않고 차례로 처리하기 때문에 매우 큰 처리 비용이 발생함<br/>

# 5.5 정리

이 장에서는 DataFrame의 기본 연산을 알아보았음<br/>
그리고 DataFrame을 사용하는 데 필요한 개념과 다양한 기능도 함께 알아보았음<br/>
다음 장에서는 DataFrame의 데이터를 다루는 다양한 방법을 자세히 알아보겠음<br/>